In [1]:
import pandas as pd

PATH = '/home/kai/data/kaggle/homecredit/'

train = pd.read_csv(PATH + 'application_train.csv')
credit = pd.read_csv(PATH + 'credit_card_balance.csv')

'done'

'done'

In [2]:
def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / (df[col2] + 1)

def positive_count(df, gp_col, col):
    group = (df[col] > 0).astype('int8')
    group = pd.concat([df[gp_col], group], axis=1).groupby(gp_col)[[col]].sum().reset_index().rename(index=str, columns={col: 'positivecount_'+'_'.join([gp_col, col])})
    return group.astype({gp_col: df[gp_col].dtype})

def count(df, gp_col, col):
    group = df[[gp_col,col]].groupby(gp_col)[[col]].count().reset_index().rename(index=str, columns={col:'count_'+gp_col})
    return group.astype({gp_col: df[gp_col].dtype})

def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index().astype({gp_col: df[gp_col].dtype})

def one_hot(df, gp_col, col):
    return pd.concat([df[gp_col], pd.get_dummies(df[col], prefix='onehot_' + col)], axis=1).groupby(gp_col).sum().reset_index().astype({gp_col: df[gp_col].dtype})

'done'

'done'

In [3]:
credit.columns

Index(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE',
       'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT',
       'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',
       'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY',
       'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT',
       'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE',
       'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT',
       'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT',
       'CNT_INSTALMENT_MATURE_CUM', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF'],
      dtype='object')

In [4]:
df = pd.DataFrame(train['SK_ID_CURR'])

merged_col = 'SK_ID_CURR'

# minus
name1 = minus_name('SK_DPD', 'SK_DPD_DEF')
credit[name1] = minus(credit, 'SK_DPD', 'SK_DPD_DEF')

# ratio
numerator = [x for x in df.columns if 'AMT' in x] + ['SK_DPD', 'SK_DPD_DEF', name1]
denominator = [x for x in df.columns if 'CNT' in x] + numerator + ['MONTHS_BALANCE']
for x in numerator:
    for y in denominator:
        if x != y:
            print(x, y)
            credit[ratio_name(x, y)] = ratio(credit, x, y)

# one hot encoding sum
df = df.merge(one_hot(credit, merged_col, 'NAME_CONTRACT_STATUS'), on=merged_col, how='left')

# count
df = df.merge(count(credit, merged_col, 'NAME_CONTRACT_STATUS'), on=merged_col, how='left')
count_name = [x for x in df.columns if 'count_' in x][0]

# positive count
for x in ['SK_DPD', 'SK_DPD_DEF', name1]:
    print(x)
    df = df.merge(positive_count(credit, merged_col, x))
    
# categorical ratio
for x in df.columns:
    if 'positivecount_' in x or 'onehot_' in x:
        print(x)
        df[ratio_name(x, count_name)] = ratio(df, x, count_name)

# numerical
ignored = ['SK_ID_CURR', 'SK_ID_PREV', 'NAME_CONTRACT_STATUS', 'MONTHS_BALANCE']
for x in set(credit.columns) - set(ignored):
    print(x)
    df = df.merge(numerical(credit, merged_col, x, ['mean', 'median', 'max', 'min', 'sum']), on=merged_col, how='left')

'done'

SK_DPD SK_DPD_DEF
SK_DPD SK_DPD_minus_SK_DPD_DEF
SK_DPD MONTHS_BALANCE
SK_DPD_DEF SK_DPD
SK_DPD_DEF SK_DPD_minus_SK_DPD_DEF
SK_DPD_DEF MONTHS_BALANCE
SK_DPD_minus_SK_DPD_DEF SK_DPD
SK_DPD_minus_SK_DPD_DEF SK_DPD_DEF
SK_DPD_minus_SK_DPD_DEF MONTHS_BALANCE
SK_DPD
SK_DPD_DEF
SK_DPD_minus_SK_DPD_DEF
onehot_NAME_CONTRACT_STATUS_Active
onehot_NAME_CONTRACT_STATUS_Approved
onehot_NAME_CONTRACT_STATUS_Completed
onehot_NAME_CONTRACT_STATUS_Demand
onehot_NAME_CONTRACT_STATUS_Refused
onehot_NAME_CONTRACT_STATUS_Sent proposal
onehot_NAME_CONTRACT_STATUS_Signed
positivecount_SK_ID_CURR_SK_DPD
positivecount_SK_ID_CURR_SK_DPD_DEF
positivecount_SK_ID_CURR_SK_DPD_minus_SK_DPD_DEF
SK_DPD_minus_SK_DPD_DEF_divide_SK_DPD_DEF
CNT_DRAWINGS_ATM_CURRENT
AMT_PAYMENT_CURRENT
AMT_RECEIVABLE_PRINCIPAL
AMT_TOTAL_RECEIVABLE
SK_DPD_DEF_divide_SK_DPD_minus_SK_DPD_DEF
CNT_DRAWINGS_CURRENT
AMT_DRAWINGS_ATM_CURRENT
AMT_BALANCE
SK_DPD_minus_SK_DPD_DEF
SK_DPD_DEF_divide_MONTHS_BALANCE
CNT_DRAWINGS_OTHER_CURRENT
CNT_DRAWING

'done'

In [5]:
print(df.columns)
df.shape

Index(['SK_ID_CURR', 'onehot_NAME_CONTRACT_STATUS_Active',
       'onehot_NAME_CONTRACT_STATUS_Approved',
       'onehot_NAME_CONTRACT_STATUS_Completed',
       'onehot_NAME_CONTRACT_STATUS_Demand',
       'onehot_NAME_CONTRACT_STATUS_Refused',
       'onehot_NAME_CONTRACT_STATUS_Sent proposal',
       'onehot_NAME_CONTRACT_STATUS_Signed', 'count_SK_ID_CURR',
       'positivecount_SK_ID_CURR_SK_DPD',
       ...
       'AMT_RECIVABLE_mean', 'AMT_RECIVABLE_median', 'AMT_RECIVABLE_max',
       'AMT_RECIVABLE_min', 'AMT_RECIVABLE_sum',
       'SK_DPD_DEF_divide_SK_DPD_mean', 'SK_DPD_DEF_divide_SK_DPD_median',
       'SK_DPD_DEF_divide_SK_DPD_max', 'SK_DPD_DEF_divide_SK_DPD_min',
       'SK_DPD_DEF_divide_SK_DPD_sum'],
      dtype='object', length=167)


(86905, 167)

In [6]:
df.to_pickle(PATH + 'inter/credit2curr.pkl')
'done'

'done'